# 모델 평가와 하이퍼파라미터 튜닝에 관한 사례학습

## 파이프라인을 가진 스트리밍 워크플로우

### Breast Cancer Wisconsin 데이터 로딩
UCI 웹사이트에서 데이터를 읽어들인다.
데이터의 앞에서부터 두 번째까지 열은 각 샘플의 ID 숫자와 이들의 진단결과(M=악성,B=양성) 값을 저장하고 있다.
3번째에서 32번째 열은 30개의 실제 값으로 이루어진 피처인데, 이들은 세포핵을 디지털화한 이미지로부터 계산된것으로 암세포가 악성인지 양성인지 예측하는 모델을 만드는 데 사용할 수 있다.

In [6]:
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header=None)


Numpy 배열 X에 30개의 피처를 할당한다.
악성(M)종양은 분류 1으로, 양성(B)종양은 분류 0으로 표기된다.

In [7]:
from sklearn.preprocessing import LabelEncoder
X = df.loc[:, 2:].values
y = df.loc[:, 1].values
le = LabelEncoder()
y = le.fit_transform(y)


훈련 데이터(데이터의 80%)와 테스트 데이터(데이터의 20%)로 나눈다.

In [5]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.20, random_state=1)

### 파이프라인 내의 변환기와 에스티메이터 결합하기
훈련 데이터와 테스트 데이터를 별도로 피팅하고 변환하는 단계를 밟는 대신, 파이프라인에 StandardScaler, PCA, LogisticRegression 오브젝트들을 연결시켜 둘 수 있다.

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipe_lr = Pipeline([('scl', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', LogisticRegression(random_state=1))])
pipe_lr.fit(X_train, y_train)
print('Test Accuracy: %.3f' % pipe_lr.score(X_test, y_test))

Test Accuracy: 0.947


Pipeline 오브젝트는 입력값으로 tuple 목록을 취한다. 여기서 각각의 tuple 내의 첫 번째 값은 임의의 식별 문자로 파이프라인에서 개별 요소에 접근하기 위해 사용할 수 있다. 모든 tuple 내의 두 번째 요소는 사이킷런 변환기 혹은 에스티메이터가 된다.